In [1]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Lattice, Simulation, Molecule
from flowermd.library import FF_from_file
import unyt as u
import hoomd
import glob
from flowermd.base.system import Pack
import mbuild as mb

cpu = hoomd.device.CPU()

# Rerun with the original seed first

In [2]:
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

In [3]:
directory = "/home/jbieri/CME_lab/repos/pl-validation/mol2/polymers/"
poly_ff_list = []
polymers_molecule_list = glob.glob(directory+"*.mol2")
for i in range(len(polymers_molecule_list)):
    mol_name = polymers_molecule_list[i][:-12].removeprefix(directory)
    poly_ff_list.append((str(polymers_molecule_list[i][:41]) + "xml/" + mol_name + ".xml"))

In [10]:
system_file = '/home/jbieri/CME_lab/repos/pl-validation/mol2/PCPDTFBT_C1_BO.mol2'
ff_filepath = '/home/jbieri/CME_lab/repos/pl-validation/xml/PCPDTFBT_C1_BO.xml'

espmol = espaloma_mol(system_file)
molecule = Molecule(num_mols=1, compound=espmol)

molff = FF_from_file(ff_filepath)
system = Pack(molecules=molecule,density=0.0001 * u.g/u.cm**3,packing_expand_factor=5)
system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
system.remove_hydrogens

No charged group detected, skipping electrostatics.


<bound method System.remove_hydrogens of <flowermd.base.system.Pack object at 0x7f4390e79e10>>

In [24]:
print(espmol)

<Compound 123 particles, 127 bonds, non-periodic, id: 139927832176208>


In [11]:
system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [16]:
system.hoomd_snapshot
hoomd_forces = system.hoomd_forcefield
hoomd_forces
lj_force = hoomd_forces[3]
cpu = hoomd.device.CPU()

In [14]:
sim = Simulation.from_system(system=system, gsd_write_freq=100, log_write_freq=100, device=cpu, gsd_file_name='melt.gsd')
sim.run_NVT(n_steps=2e4, kT=5.0, tau_kt=1.0)
sim.flush_writers()

Initializing simulation state from a gsd.hoomd.Frame.
Step 100 of 20000; TPS: 95.53; ETA: 3.5 minutes
Step 200 of 20000; TPS: 189.14; ETA: 1.7 minutes
Step 300 of 20000; TPS: 161.24; ETA: 2.0 minutes
Step 400 of 20000; TPS: 213.81; ETA: 1.5 minutes
Step 500 of 20000; TPS: 266.0; ETA: 1.2 minutes
Step 600 of 20000; TPS: 317.79; ETA: 1.0 minutes
Step 700 of 20000; TPS: 368.96; ETA: 0.9 minutes
Step 800 of 20000; TPS: 419.8; ETA: 0.8 minutes
Step 900 of 20000; TPS: 470.17; ETA: 0.7 minutes
Step 1000 of 20000; TPS: 520.19; ETA: 0.6 minutes
Step 1100 of 20000; TPS: 569.47; ETA: 0.6 minutes
Step 1200 of 20000; TPS: 618.2; ETA: 0.5 minutes
Step 1300 of 20000; TPS: 666.6; ETA: 0.5 minutes
Step 1400 of 20000; TPS: 714.88; ETA: 0.4 minutes
Step 1500 of 20000; TPS: 762.19; ETA: 0.4 minutes
Step 1600 of 20000; TPS: 809.68; ETA: 0.4 minutes
Step 1700 of 20000; TPS: 856.55; ETA: 0.4 minutes
Step 1800 of 20000; TPS: 903.12; ETA: 0.3 minutes
Step 1900 of 20000; TPS: 948.8; ETA: 0.3 minutes
Step 2000 o

In [31]:
directory = "/home/jbieri/CME_lab/repos/pl-validation/mol2/"
ff_list = []
mol_names = []
molecule_list = glob.glob(directory+"*.mol2")
for i in range(len(molecule_list)):
    mol_names.append(molecule_list[i][:-5].removeprefix(directory))
    ff_list.append((str(molecule_list[i][:41]) + "xml/" + mol_names[i] + ".xml"))

In [32]:
print(molecule_list[0])
print(ff_list[0])

/home/jbieri/CME_lab/repos/pl-validation/mol2/PCPDTPT_nC16.mol2
/home/jbieri/CME_lab/repos/pl-validation/xml/PCPDTPT_nC16.xml


In [35]:
system_file = '/home/jbieri/CME_lab/repos/pl-validation/mol2/PCPDTPT_nC16.mol2'
ff_filepath = '/home/jbieri/CME_lab/repos/pl-validation/xml/PCPDTPT_nC16.xml'

for i in range(len(molecule_list)):
    system_file = molecule_list[i]
    ff_filepath = ff_list[i]
    espmol = espaloma_mol(system_file)
    molecule = Molecule(num_mols=1, compound=espmol)
    molff = FF_from_file(ff_filepath)
    system = Pack(molecules=molecule,density=0.0001 * u.g/u.cm**3,packing_expand_factor=5)
    system.apply_forcefield(r_cut=2.5, force_field=molff, auto_scale=True,remove_charges=True, remove_hydrogens=True)
    sim = Simulation.from_system(system=system, gsd_write_freq=10, log_write_freq=100, device=cpu, gsd_file_name=(mol_names[i]+'_melt.gsd'))
    sim.run_NVT(n_steps=1e2, kT=5.0, tau_kt=1.0)
    sim.flush_writers()

No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 55 of 100; TPS: 82.03; ETA: 0.0 minutes
No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 55 of 100; TPS: 54.35; ETA: 0.0 minutes
No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 55 of 100; TPS: 45.62; ETA: 0.0 minutes
No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 55 of 100; TPS: 36.78; ETA: 0.0 minutes
No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 55 of 100; TPS: 84.6; ETA: 0.0 minutes
No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.hoomd.Frame.
Step 55 of 100; TPS: 74.57; ETA: 0.0 minutes
No charged group detected, skipping electrostatics.
Initializing simulation state from a gsd.ho